In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="lBm4iE4T4ZnkvuavmAWR")
project = rf.workspace("saharat-yyrhu").project("swimmingxdrowning")
version = project.version(4)
dataset = version.download("tensorflow")

In [ ]:
import tensorflow as tf

target_height, target_width = 224, 224


def load_and_preprocess_image(image_path):
    # Load image and resize
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [target_height, target_width])
    # Normalize image to range [0, 1]
    img = img / 255.0
    return img

In [ ]:
import pandas as pd


def get_data(path, batch_size=20):
    # Step 1: Parse the CSV file
    annotations_df = pd.read_csv(path + '_annotations.csv')

    # Step 2: Create a TensorFlow Dataset for images
    image_paths = annotations_df['filename'].tolist()
    image_paths = [path + p for p in image_paths]
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)

    # Apply preprocessing to image dataset
    image_dataset = image_dataset.map(load_and_preprocess_image)

    bounding_boxes = annotations_df[['xmin', 'ymin', 'xmax', 'ymax']].astype('float32').values
    class_labels = annotations_df['class'].apply(lambda x: 0 if x == 'Swimming' else 0).values

    # Create datasets for bounding boxes and class labels
    bounding_boxes_dataset = tf.data.Dataset.from_tensor_slices(bounding_boxes)
    class_labels_dataset = tf.data.Dataset.from_tensor_slices(class_labels)

    # Step 5: Combine image dataset with bounding boxes and class labels
    temp_dict = {
        'bounding_boxes': bounding_boxes_dataset,
        'class_labels': class_labels_dataset,
    }
    data = tf.data.Dataset.zip((image_dataset, temp_dict))

    data = (
        data.shuffle(buffer_size=1000)
        .batch(batch_size)
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    )
    return data

In [ ]:
train_dataset = get_data('SwimmingXDrowning-4/train/')

In [ ]:
train_dataset

In [ ]:

CLASSES = 2
input_size = 224


def build_classifier(inputs):
    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2, 2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D(2, 2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D(2, 2)(x)

    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(64, activation='relu')(x)

    x = tf.keras.layers.Dense(CLASSES, activation='softmax')(x)

    return x


def build_regressor(inputs):
    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2, 2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D(2, 2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D(2, 2)(x)

    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(64, activation='relu')(x)

    x = tf.keras.layers.Dense(units='4')(inputs)(x)

    return x

In [ ]:
def build_feature_extractor(inputs):

    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    return x

def build_model_adaptor(inputs):
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    return x

def build_classifier_head(inputs):
    return tf.keras.layers.Dense(CLASSES, activation='softmax', name = 'classifier_head')(inputs)

def build_regressor_head(inputs):
    return tf.keras.layers.Dense(units = '4', name = 'regressor_head')(inputs)

def build_model(inputs):
    
    feature_extractor = build_feature_extractor(inputs)

    model_adaptor = build_model_adaptor(feature_extractor)

    classification_head = build_classifier_head(model_adaptor)

    regressor_head = build_regressor_head(model_adaptor)

    model = tf.keras.Model(inputs = inputs, outputs = [classification_head, regressor_head])

    return model

In [ ]:
model = build_model(tf.keras.layers.Input(shape=(input_size, input_size, 1,)))

model.compile(optimizer=tf.keras.optimizers.Adam(), 
    loss = {'classifier_head' : 'categorical_crossentropy', 'regressor_head' : 'mse' }, 
    metrics = {'classifier_head' : 'accuracy', 'regressor_head' : 'mse' })
    
EPOCHS = 10
BATCH_SIZE = 32

history = model.fit(train_dataset, epochs=EPOCHS)